Notebook SUPREMOOOOO

In [6]:
#versione mery

import numpy as np
from ultralytics import YOLO
import os
from pathlib import Path
import cv2

base_dir = Path().resolve()
os.chdir(base_dir)

model = YOLO("3.weights/best.pt")

classes_fd = model.names
print(f"Face detection model is able to detect following classes:\n{classes_fd}")

base_dir = Path().resolve()
os.chdir(base_dir)

Face detection model is able to detect following classes:
{0: 'BABY', 1: 'EYE', 2: 'HEAD', 3: 'MOUTH', 4: 'NOSE'}


# DATASET ON BACK/ON STOMACH: 2
Extract dictionary {img_name : label} from annotations.json file

In [7]:
import json

file_imgid = {}
imgid_label = {}
file_label = {}

classes_mlp = {}

with open("onback_onstomach_v2/_annotations.coco.json", "r") as f:
    dataset = json.load(f)
    
    for img in dataset["images"]:
        file_imgid[img["file_name"]] =img["id"]
    
    for label in dataset["annotations"]:
        imgid_label[label["image_id"]] = label["category_id"]
    
    for cls in dataset["categories"]:
        classes_mlp[cls["name"]]=cls["id"]
    del classes_mlp["baby"]

for file, img_id in file_imgid.items():
    if img_id in imgid_label.keys():
        file_label[file] = imgid_label[img_id]      

# Save new dataset (embedding - label) in .csv format

In [8]:
import csv

def save_embedding_label_dataset(dataset: str, embedding_type: str, model_version:str, X, y):    
    with open(f"{dataset}/{model_version}-{embedding_type}.csv", "w", newline='') as f:
        writer = csv.writer(f)
        for xi, yi in zip(X, y):
            writer.writerow(list(xi) + [yi])

# Load dataset from .csv file

In [9]:
def load_embedding_label_dataset(dataset: str, embedding_type: str, model_version: str):
    data = np.loadtxt(f"{dataset}/{model_version}-{embedding_type}.csv", delimiter=",")
    X = data[:, :-1]
    y = data[:, -1]
    
    return X,y

# COMBINATION 1 (dataset 2, model 2, embedding SUPREMOOO)

In [14]:
model_version = "v3"
dataset_folder = "onback_onstomach_v2"
embedding_type = "supremo"

In [17]:
import embeddings 
onback_onstomach_dataset = Path(dataset_folder)
X = []
y = []

for img in onback_onstomach_dataset.glob("*.jpg"):
    if img.name in file_label.keys():
        #DEBUG
        if len(y)%100 == 0:
            print(f"{int(len(y)*100/len(file_label))}%-->    {len(y)} / {len(file_label)} embeddings created")
        #DEBUG    
        result = model(img, conf=0.3, verbose=False)[0]
        prediction = result.boxes
        
        vec = embeddings.embedding_SUPREMO(prediction)
        X.append(vec)
        y.append(file_label[img.name])
        
print(f"FINISHED EMBEDDING CREATION, {len(y)} embeddings created")

save_embedding_label_dataset(dataset_folder, embedding_type, model_version, X, y)
print(f"New dataset (embedding - label) saved in '{dataset_folder}/{embedding_type}.csv'")

SyntaxError: invalid syntax (embeddings.py, line 236)

X, y = load_embedding_label_dataset(dataset_folder, embedding_type, model_version)

# RANDOM FOREST con feature selection

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import matplotlib.pyplot as plt

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Ottieni l'importanza delle feature
importances = clf.feature_importances_

# Ordina per importanza decrescente
indices = np.argsort(importances)[::-1]

# Visualizza le top 10 feature
plt.figure(figsize=(10, 5))
plt.title("Top 10 Feature Importances")
plt.bar(range(10), importances[indices[:10]])
plt.xticks(range(10), [f"Feature {i}" for i in indices[:10]], rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import classification_report
# Prendi solo le prime 10 feature
top_k = 10
top_features_idx = indices[:top_k]

# Filtra X_train e X_test per includere solo le top-k feature
X_train_selected = X_train[:, top_features_idx]
X_test_selected = X_test[:, top_features_idx]

clf_reduced = RandomForestClassifier(n_estimators=100, random_state=42)
clf_reduced.fit(X_train_selected, y_train)

y_pred_reduced = clf_reduced.predict(X_test_selected)


# Evaluate
print(f"\nDataset labels:----------------------------------------\n{classes_mlp}\n")

print(f"Report-------------------------------------------------")
labels_in_dataset = list(classes_mlp.keys())[:2]
print(classification_report(y_test, y_pred_reduced, target_names=labels_in_dataset))